# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: - - .

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: - - - <текст статьи>

In [19]:
# создадим пустой список новостей
news_list = list()
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
#будем вычленять части 'div', class_='post__text post__text-html post__text_v2'
posts = soup.find_all('div', class_='post__text post__text-html post__text_v2')

#ключевые слова
#KEYWORDS = ['python', 'парсинг']
#Ничего не находит, на всякий случай найдем другие слова
KEYWORDS = ['статья', 'приложение', 'при']

# извлекаем посты и записываем в список news_list
for post in posts:
    for words in KEYWORDS:
        if words in post.text:
            news_list.append(post.text)
            
#делаем датафрейм из списка
habr_news = pd.DataFrame(news_list)
habr_news

,0
0,"\nНебольшая коллекция практик, трюков и подска..."
1,"\nЗдравствуйте, это вторая статья о NGFW решен..."
2,"\nПривет, Хабр, я член Brmlab Hackerspace в Пр..."


## Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: - <дата утечки> - <источник утечки> - <описание утечки>.

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [3]:
import json
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast'
  }
#data = {'emailAddresses': ['xxx@x.ru', 'lol@lol.ru']}
data = {'emailAddresses': ['spb2010@mail.ru', 'lefso@bk.ru']}
json_data = json.dumps(data)
response = requests.post(url, data=json_data, headers=headers, verify=False)

C:\Program Files\Anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'identityprotection.avast.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
re_ = json.loads(response.text)
lol1 = list()
lol2 = list()
lol3 = list()

#response.text
ree_ = re_['breaches']
for keyss in ree_.keys():
    #print(ree_[keyss]['publishDate'])
    lol1.append(ree_[keyss]['publishDate'])
    #print(ree_[keyss]['site'])
    lol2.append(ree_[keyss]['site']) 
    lol3.append(ree_[keyss]['description'])
    #print(ree_[keyss]['description'])
    
kom_news = pd.DataFrame(lol1)
kom_news['site'] = pd.DataFrame(lol2)
kom_news['description'] = pd.DataFrame(lol3)
kom_news

,0,site,description
0,2019-03-07T00:00:00Z,armorgames.com,"In December 2018, Armor Games' database was al..."
1,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
2,2018-01-20T00:00:00Z,couponmom.com / armor games,"In 2014, this list of records surfaced online ..."
3,2020-01-23T00:00:00Z,kotofoto.ru,"In November 2019, the Russian online electroni..."
4,2018-06-19T00:00:00Z,storelp.ru,"At an unconfirmed date, StoreLP.ru's database ..."
5,2020-07-16T00:00:00Z,unknown,"At an unconfirmed date, data purportedly assoc..."


## Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.